In [1]:
import pandas as pd
csv_path = '../csv/BTC/KRW_BTC_day.csv'
df = pd.read_csv(csv_path)

df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.set_index('Unnamed: 0', inplace=True)
df.columns = df.columns.str.capitalize()

In [2]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
bt = Backtest(df, SmaCross, cash=1000000000, commission=.002)
stats = bt.run()
stats

Start                     2023-05-02 09:00:00
End                       2024-05-23 09:00:00
Duration                    387 days 00:00:00
Exposure Time [%]                   91.494845
Equity Final [$]                 2022573322.0
Equity Peak [$]                  2236291754.0
Return [%]                         102.257332
Buy & Hold Return [%]              147.103242
Return (Ann.) [%]                   93.986172
Volatility (Ann.) [%]               79.516091
Sharpe Ratio                         1.181977
Sortino Ratio                         3.87116
Calmar Ratio                         4.580623
Max. Drawdown [%]                  -20.518209
Avg. Drawdown [%]                   -4.391321
Max. Drawdown Duration      131 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                   17
Win Rate [%]                        41.176471
Best Trade [%]                      65.036593
Worst Trade [%]                    -12.160349
Avg. Trade [%]                    